# Análise de Dados de Sepse na MIMIC-IV

## Descrição do Projeto
Este notebook visa identificar suspeitas de infecção em pacientes sépticos utilizando a base de dados MIMIC-IV. A análise é realizada em várias etapas, combinando dados de prescrição de antibióticos e eventos de microbiologia para determinar se há suspeita de infecção e o momento da mesma.

In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Preparação dos Dados

### Criação de novos dataframes *antibiotic_data* e *microbiology_data*
Foram utilizadas as tabelas *prescriptions* e *microbiology_events* para criar novos dataframes *antibiotic_data* e *microbiology_data* para facilitar a leitura e manipulação dos dados

In [99]:
# URLs dos arquivos CSV no GitHub
URL_BASES = "https://raw.githubusercontent.com/victoriafloresdev/sepsis-clustering/main/Databases/MIMIC/"

# Nomes dos arquivos CSV
csv_files = {
    'admissions': 'admissions.csv',
    'patients': 'patients.csv',
    'transfers': 'transfers.csv',
    'd_icd_diagnoses': 'd_icd_diagnoses.csv',
    'd_icd_procedures': 'd_icd_procedures.csv',
    'd_labitems': 'd_labitems.csv',
    'diagnoses_icd': 'diagnoses_icd.csv',
    'labevents': 'labevents.csv',
    'microbiologyevents': 'microbiologyevents.csv',
    'omr': 'omr.csv',
    'pharmacy': 'pharmacy.csv',
    'poe_detail': 'poe_detail.csv',
    'poe': 'poe.csv',
    'prescriptions': 'prescriptions.csv',
    'icustays' : 'icustays.csv'
}

# Carregar dados em DataFrames
dfs = {}
for name, file in csv_files.items():
    url = f"{URL_BASES}{file}"
    dfs[name] = pd.read_csv(url)
    dfs[name].name = name

# Exibir as primeiras linhas de cada DataFrame
for name, df in dfs.items():
    print(f"Tabela: {df.name}")
    print(df.head())
    print()


Tabela: admissions
   subject_id   hadm_id            admittime            dischtime  \
0    10004235  24181354  2196-02-24 14:38:00  2196-03-04 14:02:00   
1    10009628  25926192  2153-09-17 17:08:00  2153-09-25 13:20:00   
2    10018081  23983182  2134-08-18 02:02:00  2134-08-23 19:35:00   
3    10006053  22942076  2111-11-13 23:39:00  2111-11-15 17:20:00   
4    10031404  21606243  2113-08-04 18:46:00  2113-08-06 20:57:00   

             deathtime admission_type admit_provider_id  \
0                  NaN         URGENT            P03YMR   
1                  NaN         URGENT            P41R5N   
2                  NaN         URGENT            P233F6   
3  2111-11-15 17:20:00         URGENT            P38TI6   
4                  NaN         URGENT            P07HDB   

       admission_location        discharge_location insurance language  \
0  TRANSFER FROM HOSPITAL  SKILLED NURSING FACILITY  Medicaid  ENGLISH   
1  TRANSFER FROM HOSPITAL          HOME HEALTH CARE  Medicaid  

In [100]:
# Definir a lista de antibióticos
antibiotics_list = [
    'adoxa', 'ala-tet', 'alodox', 'amikacin', 'amikin', 'amoxicill', 'amphotericin', 'anidulafungin',
    'ancef', 'clavulanate', 'ampicillin', 'augmentin', 'avelox', 'avidoxy', 'azactam', 'azithromycin',
    'aztreonam', 'axetil', 'bactocill', 'bactrim', 'bactroban', 'bethkis', 'biaxin', 'bicillin l-a',
    'cayston', 'cefazolin', 'cedax', 'cefoxitin', 'ceftazidime', 'cefaclor', 'cefadroxil', 'cefdinir',
    'cefditoren', 'cefepime', 'cefotan', 'cefotetan', 'cefotaxime', 'ceftaroline', 'cefpodoxime', 'cefpirome',
    'cefprozil', 'ceftibuten', 'ceftin', 'ceftriaxone', 'cefuroxime', 'cephalexin', 'cephalothin', 'cephapririn',
    'chloramphenicol', 'cipro', 'ciprofloxacin', 'claforan', 'clarithromycin', 'cleocin', 'clindamycin', 'cubicin',
    'dicloxacillin', 'dirithromycin', 'doryx', 'doxycy', 'duricef', 'dynacin', 'ery-tab', 'eryped', 'eryc',
    'erythrocin', 'erythromycin', 'factive', 'flagyl', 'fortaz', 'furadantin', 'garamycin', 'gentamicin',
    'kanamycin', 'keflex', 'kefzol', 'ketek', 'levaquin', 'levofloxacin', 'lincocin', 'linezolid', 'macrobid',
    'macrodantin', 'maxipime', 'mefoxin', 'metronidazole', 'meropenem', 'methicillin', 'minocin', 'minocycline',
    'monodox', 'monurol', 'morgidox', 'moxatag', 'moxifloxacin', 'mupirocin', 'myrac', 'nafcillin', 'neomycin',
    'nicazel doxy 30', 'nitrofurantoin', 'norfloxacin', 'noroxin', 'ocudox', 'ofloxacin', 'omnicef', 'oracea',
    'oraxyl', 'oxacillin', 'pc pen vk', 'pce dispertab', 'panixine', 'pediazole', 'penicillin', 'periostat',
    'pfizerpen', 'piperacillin', 'tazobactam', 'primsol', 'proquin', 'raniclor', 'rifadin', 'rifampin', 'rocephin',
    'smz-tmp', 'septra', 'septra ds', 'septra', 'solodyn', 'spectracef', 'streptomycin', 'sulfadiazine',
    'sulfamethoxazole', 'trimethoprim', 'sulfatrim', 'sulfisoxazole', 'suprax', 'synercid', 'tazicef',
    'tetracycline', 'timentin', 'tobramycin', 'trimethoprim', 'unasyn', 'vancocin', 'vancomycin', 'vantin',
    'vibativ', 'vibra-tabs', 'vibramycin', 'zinacef', 'zithromax', 'zosyn', 'zyvox'
]

# Criar DataFrame de antibióticos
antibiotics_df = pd.DataFrame({'drug': antibiotics_list})

# Carregar o DataFrame de prescrições
prescriptions_df = dfs['prescriptions']

# Função para identificar se o medicamento é um antibiótico
def is_antibiotic(drug):
    drug_lower = drug.lower()
    for abx in antibiotics_list:
        if abx in drug_lower:
            return 1
    return 0

# Aplicar a função para criar a coluna de antibióticos
prescriptions_df['antibiotic'] = prescriptions_df['drug'].apply(is_antibiotic)

# Filtrar as prescrições que são antibióticos
antibiotic_df = prescriptions_df[prescriptions_df['antibiotic'] == 1]

# Adicionar o DataFrame de antibióticos ao dicionário dfs
dfs['antibiotics'] = antibiotics_df

# Exibir as primeiras linhas do novo DataFrame de antibióticos
print("Primeiras linhas do DataFrame de Antibióticos:")
print(antibiotic_df.head())

Primeiras linhas do DataFrame de Antibióticos:
     subject_id   hadm_id  pharmacy_id         poe_id  poe_seq  \
293    10015860  25103777     19011988  10015860-1341   1341.0   
299    10038992  22797747     77148582    10038992-48     48.0   
551    10014078  25809882     58803034    10014078-73     73.0   
552    10027602  28166872     67096778   10027602-276    276.0   
553    10027602  28166872     91079440   10027602-331    331.0   

    order_provider_id            starttime             stoptime drug_type  \
293            P13B4S  2192-07-31 22:00:00  2192-08-01 08:00:00      MAIN   
299            P61UOA  2185-11-02 21:00:00  2185-11-08 20:00:00      MAIN   
551            P326AV  2166-08-22 10:00:00  2166-08-24 23:00:00      MAIN   
552            P91TFA  2201-11-08 11:00:00  2201-11-11 09:00:00      MAIN   
553            P42Q7N  2201-11-11 12:00:00  2201-11-12 09:00:00      MAIN   

                  drug  ...           ndc prod_strength  form_rx dose_val_rx  \
293         V

In [101]:
# Tabela de dados de antibióticos
antibiotic_data = antibiotic_df[[
    'subject_id', 'hadm_id', 'starttime', 'stoptime', 'drug', 'pharmacy_id'
]].rename(columns={'drug': 'antibiotic'})

# Tabela de dados de microbiologia
microbiology_data = dfs['microbiologyevents'][[
    'subject_id', 'hadm_id', 'chartdate', 'charttime', 'spec_type_desc', 'org_name', 'org_itemid'
]]

# Identificar culturas positivas
microbiology_data['positiveculture'] = microbiology_data.apply(
    lambda row: 1 if pd.notnull(row['org_name']) and row['org_itemid'] != 90856 and row['org_name'] != '' else 0,
    axis=1
)

# Salvar arquivos manipulados
antibiotic_data.to_csv('antibiotic_data.csv', index=False)
microbiology_data.to_csv('microbiology_data.csv', index=False)

/tmp/ipykernel_246144/682179113.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microbiology_data['positiveculture'] = microbiology_data.apply(


In [87]:
# Exibir as primeiras linhas dos dados de antibióticos
antibiotic_data.head()

,subject_id,hadm_id,starttime,stoptime,antibiotic,pharmacy_id
293,10015860,25103777,2192-07-31 22:00:00,2192-08-01 08:00:00,Vancomycin,19011988
299,10038992,22797747,2185-11-02 21:00:00,2185-11-08 20:00:00,vancomycin,77148582
551,10014078,25809882,2166-08-22 10:00:00,2166-08-24 23:00:00,Ampicillin Sodium,58803034
552,10027602,28166872,2201-11-08 11:00:00,2201-11-11 09:00:00,Ampicillin Sodium,67096778
553,10027602,28166872,2201-11-11 12:00:00,2201-11-12 09:00:00,Ampicillin Sodium,91079440


In [88]:
# Exibir as primeiras linhas dos dados de microbiologia
microbiology_data.head()

,subject_id,hadm_id,chartdate,charttime,spec_type_desc,org_name,org_itemid,positiveculture
0,10000032,25742920.0,2180-08-06 00:00:00,2180-08-06 20:35:00,SWAB,NaN,NaN,0
1,10000032,22595853.0,2180-05-07 00:00:00,2180-05-07 00:19:00,SWAB,NaN,NaN,0
2,10000032,29079034.0,2180-07-24 00:00:00,2180-07-24 00:55:00,SWAB,NaN,NaN,0
3,10020944,29974575.0,2131-02-27 00:00:00,2131-02-27 17:41:00,SWAB,ENTEROCOCCUS SP.,80053.0,1
4,10037975,27617929.0,2185-01-17 00:00:00,2185-01-17 21:32:00,SWAB,NaN,NaN,0


### Identificadores de Tratamento de Antibióticos
Foram criados identificadores únicos para cada tratamento de antibiótico utilizando a combinação do `subject_id` e `starttime`.

### Ajuste dos Tipos de Dados
As colunas de datas foram convertidas para o tipo `datetime` para facilitar as operações de comparação de datas.

In [102]:
# Criar identificadores únicos para cada tratamento de antibiótico
antibiotic_data['ab_id'] = antibiotic_data.groupby(['subject_id', 'starttime']).ngroup()

# Ajustar os tipos de dados para datas
antibiotic_data['starttime'] = pd.to_datetime(antibiotic_data['starttime'])
antibiotic_data['stoptime'] = pd.to_datetime(antibiotic_data['stoptime'])
microbiology_data['chartdate'] = pd.to_datetime(microbiology_data['chartdate'])
microbiology_data['charttime'] = pd.to_datetime(microbiology_data['charttime'])

# Exibir as primeiras linhas dos dados ajustados
antibiotic_data.head()
microbiology_data.head()


/tmp/ipykernel_246144/394726720.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microbiology_data['chartdate'] = pd.to_datetime(microbiology_data['chartdate'])
/tmp/ipykernel_246144/394726720.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microbiology_data['charttime'] = pd.to_datetime(microbiology_data['charttime'])


,subject_id,hadm_id,chartdate,charttime,spec_type_desc,org_name,org_itemid,positiveculture
0,10000032,25742920.0,2180-08-06,2180-08-06 20:35:00,SWAB,NaN,NaN,0
1,10000032,22595853.0,2180-05-07,2180-05-07 00:19:00,SWAB,NaN,NaN,0
2,10000032,29079034.0,2180-07-24,2180-07-24 00:55:00,SWAB,NaN,NaN,0
3,10020944,29974575.0,2131-02-27,2131-02-27 17:41:00,SWAB,ENTEROCOCCUS SP.,80053.0,1
4,10037975,27617929.0,2185-01-17,2185-01-17 21:32:00,SWAB,NaN,NaN,0


## Identificação de Culturas Microbiológicas

### Culturas Positivas
Foi criada uma coluna para identificar culturas microbiológicas positivas e o df *relevant_cultures* armazena as culturas relevantes para análise posterior.

In [103]:
# Criar coluna para identificar culturas positivas
microbiology_data['positiveculture'] = np.where(microbiology_data['org_name'].notna(), 1, 0)

# Filtrar culturas relevantes
relevant_cultures = microbiology_data[microbiology_data['positiveculture'] == 1]

# Exibir as primeiras linhas das culturas relevantes
relevant_cultures.head()


/tmp/ipykernel_246144/1533957248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microbiology_data['positiveculture'] = np.where(microbiology_data['org_name'].notna(), 1, 0)


,subject_id,hadm_id,chartdate,charttime,spec_type_desc,org_name,org_itemid,positiveculture
3,10020944,29974575.0,2131-02-27,2131-02-27 17:41:00,SWAB,ENTEROCOCCUS SP.,80053.0,1
109,10001725,NaN,2113-06-03,NaT,SWAB,YEAST,80075.0,1
110,10023239,NaN,2141-06-20,NaT,SWAB,YEAST,80075.0,1
111,10001725,NaN,2110-01-04,2110-01-04 14:33:00,SWAB,YEAST,80075.0,1
178,10020740,23831430.0,2150-03-19,2150-03-19 06:08:00,MRSA SCREEN,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,80293.0,1


## Definição de Suspeita de Infecção

### Verificação de Suspeita de Infecção
Para cada tratamento de antibiótico, foi verificado se há culturas microbiológicas positivas nas 72 horas antes ou nas 24 horas após a administração do antibiótico. Se qualquer uma dessas condições for atendida, o paciente é marcado com suspeita de infecção.

In [104]:
# Função para verificar suspeita de infecção
def check_suspected_infection(antibiotic_row):
    subject_id = antibiotic_row['subject_id']
    ab_starttime = antibiotic_row['starttime']
    ab_stoptime = antibiotic_row['stoptime']
    
    # Verificar culturas nas 72 horas antes do antibiótico
    cultures_before = relevant_cultures[(relevant_cultures['subject_id'] == subject_id) & 
                                        (relevant_cultures['charttime'] <= ab_starttime) &
                                        (relevant_cultures['charttime'] >= ab_starttime - pd.Timedelta(hours=72))]
    
    # Verificar culturas nas 24 horas depois do antibiótico
    cultures_after = relevant_cultures[(relevant_cultures['subject_id'] == subject_id) & 
                                       (relevant_cultures['charttime'] >= ab_starttime) &
                                       (relevant_cultures['charttime'] <= ab_starttime + pd.Timedelta(hours=24))]
    
    if not cultures_before.empty or not cultures_after.empty:
        return 1
    return 0

# Aplicar a função a cada linha de antibiotic_data
antibiotic_data['suspected_infection'] = antibiotic_data.apply(check_suspected_infection, axis=1)

# Exibir as primeiras linhas dos dados com suspeita de infecção
antibiotic_data.head()


,subject_id,hadm_id,starttime,stoptime,antibiotic,pharmacy_id,ab_id,suspected_infection
293,10015860,25103777,2192-07-31 22:00:00,2192-08-01 08:00:00,Vancomycin,19011988,337,1
299,10038992,22797747,2185-11-02 21:00:00,2185-11-08 20:00:00,vancomycin,77148582,702,0
551,10014078,25809882,2166-08-22 10:00:00,2166-08-24 23:00:00,Ampicillin Sodium,58803034,260,0
552,10027602,28166872,2201-11-08 11:00:00,2201-11-11 09:00:00,Ampicillin Sodium,67096778,579,1
553,10027602,28166872,2201-11-11 12:00:00,2201-11-12 09:00:00,Ampicillin Sodium,91079440,580,1


## Seleção Final 
Foram selecionadas as colunas relevantes para a análise final, que incluem o ID do paciente, ID da hospitalização, ID da permanência na UTI, ID do tratamento de antibiótico, nome do antibiótico, hora de início do antibiótico e a indicação de suspeita de infecção.


In [105]:
# Selecionar colunas relevantes para a análise final
final_df = antibiotic_data[['subject_id', 'hadm_id', 'ab_id', 'antibiotic', 'starttime', 'suspected_infection']]

# Exibir as primeiras linhas do resultado final
final_df.head()


,subject_id,hadm_id,ab_id,antibiotic,starttime,suspected_infection
293,10015860,25103777,337,Vancomycin,2192-07-31 22:00:00,1
299,10038992,22797747,702,vancomycin,2185-11-02 21:00:00,0
551,10014078,25809882,260,Ampicillin Sodium,2166-08-22 10:00:00,0
552,10027602,28166872,579,Ampicillin Sodium,2201-11-08 11:00:00,1
553,10027602,28166872,580,Ampicillin Sodium,2201-11-11 12:00:00,1


In [93]:
# Filtrar linhas onde 'suspected_infection' é igual a 1
suspected_infection_df = final_df.loc[final_df['suspected_infection'] == 1]

# Exibir as primeiras linhas do resultado final
print(suspected_infection_df.head())

      subject_id   hadm_id  ab_id                 antibiotic  \
293     10015860  25103777    337                 Vancomycin   
552     10027602  28166872    579          Ampicillin Sodium   
553     10027602  28166872    580          Ampicillin Sodium   
1662    10007818  22987108    187                  Aztreonam   
1669    10037928  29802992    672  Sulfameth/Trimethoprim DS   

               starttime  suspected_infection  
293  2192-07-31 22:00:00                    1  
552  2201-11-08 11:00:00                    1  
553  2201-11-11 12:00:00                    1  
1662 2146-06-22 23:00:00                    1  
1669 2179-07-27 15:00:00                    1  


In [107]:

# Renomear starttime para antibiotic_time
antibiotic_data.rename(columns={'starttime': 'antibiotic_time'}, inplace=True)

# Adicionar culture_time, specimen, e positive_culture_time ao dataframe final soi
# Usar microbiology_data para culture_time e specimen
# Usar relevant_cultures para positive_culture_time
final_df['culture_time'] = np.nan
final_df['suspected_infection_time'] = np.nan
final_df['specimen'] = np.nan
final_df['positive_culture'] = np.nan

# Função para criar stay_id
icustays = dfs['icustays'][['subject_id', 'hadm_id', 'stay_id']].drop_duplicates()

# Merge para adicionar stay_id ao dataframe final
final_df = pd.merge(antibiotic_data, icustays, on=['subject_id', 'hadm_id'], how='left')

# Criar identificadores únicos para cada tratamento de antibiótico
final_df['ab_id'] = final_df['pharmacy_id']

# Iterar sobre final_df para preencher as colunas adicionadas
for index, row in final_df.iterrows():
    subject_id = row['subject_id']
    hadm_id = row['hadm_id']
    ab_id = row['ab_id']
    
    # Encontrar culture_time e specimen em microbiology_data
    cultures = microbiology_data[(microbiology_data['subject_id'] == subject_id) & (microbiology_data['hadm_id'] == hadm_id)]
    if not cultures.empty:
        final_df.loc[index, 'culture_time'] = cultures['charttime'].iloc[0]
        final_df.loc[index, 'specimen'] = cultures['org_name'].iloc[0]
    
    # Encontrar positive_culture_time em relevant_cultures
    positive_cultures = relevant_cultures[(relevant_cultures['subject_id'] == subject_id) & (relevant_cultures['hadm_id'] == hadm_id)]
    if not positive_cultures.empty:
        final_df.loc[index, 'positive_culture'] = 1
        final_df.loc[index, 'suspected_infection_time'] = positive_cultures['charttime'].iloc[0]

# Renomear a coluna starttime para antibiotic_time
final_df = final_df.rename(columns={'starttime': 'antibiotic_time'})

# Renomear final_df para soi e selecionar colunas finais
soi = final_df[['subject_id', 'stay_id', 'ab_id', 'antibiotic', 'antibiotic_time', 'culture_time', 'suspected_infection', 'suspected_infection_time', 'specimen', 'positive_culture']]


# Salvar soi como arquivo CSV
soi.to_csv('soi.csv', index=False)

soi.head()


/tmp/ipykernel_246144/1907719809.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  final_df.loc[index, 'specimen'] = cultures['org_name'].iloc[0]


,subject_id,stay_id,ab_id,antibiotic,antibiotic_time,culture_time,suspected_infection,suspected_infection_time,specimen,positive_culture
0,10015860,NaN,19011988,Vancomycin,2192-07-31 22:00:00,2192-08-02 06:32:00,1,NaT,NaN,NaN
1,10038992,NaN,77148582,vancomycin,2185-11-02 21:00:00,2185-11-02 21:30:00,0,NaT,NaN,NaN
2,10014078,38907302.0,58803034,Ampicillin Sodium,2166-08-22 10:00:00,2166-08-22 02:04:00,0,2166-08-23 11:13:00,NaN,1.0
3,10027602,32391858.0,67096778,Ampicillin Sodium,2201-11-08 11:00:00,2201-10-30 17:09:00,1,2201-11-05 02:20:00,NaN,1.0
4,10027602,32391858.0,91079440,Ampicillin Sodium,2201-11-11 12:00:00,2201-10-30 17:09:00,1,2201-11-05 02:20:00,NaN,1.0


### Lista de medicamentos classificados como antibióticos no script

1. Adoxa
2. Ala-Tet
3. Aldox
4. Amikacin
5. Amikin
6. Amoxicill
7. Amphotericin
8. Anidulafungin
9. Ancef
10. Clavulanate
11. Ampicillin
12. Augmentin
13. Avelox
14. Avidoxy
15. Azactam
16. Azithromycin
17. Aztreonam
18. Axetil
19. Bactocill
20. Bactrim
21. Bactroban
22. Bethkis
23. Biaxin
24. Bicillin L-A
25. Cayston
26. Cefazolin
27. Cedax
28. Cefoxitin
29. Ceftazidime
30. Cefaclor
31. Cefadroxil
32. Cefdinir
33. Cefditoren
34. Cefepime
35. Cefotan
36. Cefotetan
37. Cefotaxime
38. Ceftaroline
39. Cefpodoxime
40. Cefpirome
41. Cefprozil
42. Ceftibuten
43. Ceftin
44. Ceftriaxone
45. Cefuroxime
46. Cephalexin
47. Cephalothin
48. Cephapririn
49. Chloramphenicol
50. Cipro
51. Ciprofloxacin
52. Claforan
53. Clarithromycin
54. Cleocin
55. Clindamycin
56. Cubicin
57. Dicloxacillin
58. Dirithromycin
59. Doryx
60. Doxycy
61. Duricef
62. Dynacin
63. Ery-Tab
64. Eryped
65. Eryc
66. Erythrocin
67. Erythromycin
68. Factive
69. Flagyl
70. Fortaz
71. Furadantin
72. Garamycin
73. Gentamicin
74. Kanamycin
75. Keflex
76. Kefzol
77. Ketek
78. Levaquin
79. Levofloxacin
80. Lincocin
81. Linezolid
82. Macrobid
83. Macrodantin
84. Maxipime
85. Mefoxin
86. Metronidazole
87. Meropenem
88. Methicillin
89. Minocin
90. Minocycline
91. Monodox
92. Monurol
93. Morgidox
94. Moxatag
95. Moxifloxacin
96. Mupirocin
97. Myrac
98. Nafcillin
99. Neomycin
100. Nicazel Doxy 30
101. Nitrofurantoin
102. Norfloxacin
103. Noroxin
104. Ocudox
105. Ofloxacin
106. Omnicef
107. Oracea
108. Oraxyl
109. Oxacillin
110. PC Pen VK
111. PCE Dispertab
112. Panixine
113. Pediazole
114. Penicillin
115. Periostat
116. Pfizerpen
117. Piperacillin
118. Tazobactam
119. Primsol
120. Proquin
121. Raniclor
122. Rifadin
123. Rifampin
124. Rocephin
125. SMZ-TMP
126. Septra
127. Septra DS
128. Septra
129. Solodyn
130. Spectracef
131. Streptomycin
132. Sulfadiazine
133. Sulfamethoxazole
134. Trimethoprim
135. Sulfatrim
136. Sulfisoxazole
137. Suprax
138. Synercid
139. Tazicef
140. Tetracycline
141. Timentin
142. Tobramycin
143. Trimethoprim
144. Unasyn
145. Vancocin
146. Vancomycin
147. Vantin
148. Vibativ
149. Vibra-Tabs
150. Vibramycin
151. Zinacef
152. Zithromax
153. Zosyn
154. Zyvox